# GWAS COPDGene Freeze4

#### Load HAIL:

In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.0
SparkUI available at http://ip-1-0-30-3.ec2.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.18-cfb3e336c553
LOGGING: writing to /opt/hail-on-AWS-spot-instances/notebook/copd/hail-20190723-1042-0.2.18-cfb3e336c553.log


#### Load packages:

In [2]:
from hail.plot import show
hl.plot.output_notebook()
from hail.expr import aggregators 
from hail.expr.expressions import *
from hail.expr.expressions import Expression
from hail.typecheck import *
from hail import Table

from bokeh.plotting import figure, output_file, show, save
from pprint import pprint
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import Span, ColumnDataSource
output_notebook()
from bokeh.models import *

import numpy as np
from numpy import array, empty
import pandas as pd
from pprint import pprint
from math import log, isnan, log10
from itertools import cycle
import os, time, sys 

Loading BokehJS ...

Loading BokehJS ...

In [3]:
plot_path=os.getcwd()+'/ssc'
sys.path.append(plot_path)
import plotting

#### Load COPD freeze 4 matrix and annotation table

Import matrixtable with genomic data and table with phenotypic data from google storage (gs) bucket. Then, annotate the table to matrixtable.


In [4]:
mt = hl.read_matrix_table("s3://path_to_/raw")
t  = hl.import_table('s3://path_to_/COPDannotations.txt', impute = True).key_by('Samples')

mt = mt.annotate_cols(**t[mt.s]) 

COPDGene cohort is part of the bigger TOPMed initiative, as consequency each VCF file (that we visualize in MT format) has all possible SNPs from all projects within TOPMed (219.154.452). Therefore removing  monomorphic data, which are variants homozygous to reference (hom_ref) and thus keeping all other polymorphic variants (SNPs, indels, etc).

In [9]:
mt = mt.filter_rows(hl.agg.fraction(mt.GT.is_hom_ref()) < 1) 

In [ ]:
CPU = 16
nodes = 20
mt = mt.repartition(4 * CPU * nodes)

### Speed up by dividing mt in partitions
- Number of repartitions depends on CPU per node and number of nodes.
- especially PCA sensitive to partitions, important to use it before PCA

# 1. Sanity check

Alleles consist out of reference nucleotide(s) and alternate nucleotide(s). For each locus, an index is created based on all possibilities occuring in the samples. The reference is always indexed 0, and the alternates starting from 1 up to the number of possibile variants. A GT call consist out of two alleles per sample(diploid human), either corresponding to REF (0) or to index of possible ALT_n (n = 1, 2, etc).  

Show globals, rows and columns of matrixtable.

In [10]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'dbGaP_Subject_ID': int32
    'Consent_groups': str
    'Age': float64
    'Gender': str
    'is_female': int32
    'Race': str
    'Affection': str
    'AffectionBool': int32
    'Heart_Rate': str
    'SaO2': str
    'Images': str
    'CurrentSmoker': str
    'SmokerBool': int32
    'PackYear': str
    'FEV1PB': str
    'FEV1FVC': str
    'FEV1perc': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        DP: int32, 
        AVGDP: float64, 
        GC: array<int32>, 
        GN: int32, 
        HWEAF: array<float64>, 
        HWDGF: array<float64>, 
        IBC: float64, 
        HWE_SLP: float64, 
        ABE: float64, 
        ABZ:

In [ ]:
mt.count()

Alleles consist out of reference nucleotide(s) and alternate nucleotide(s). For each locus, an index is created based on all possibilities occuring in the samples. The reference is always indexed 0, and the alternates starting from 1 up to the number of possibile variants. A GT call consist out of two alleles per sample(diploid human), either corresponding to REF (0) or to index of possible ALT_n (n = 1, 2, etc). Because this data is bi-allelic, we find 0 and 1. 

In [ ]:
mt.row.show(5)

In [ ]:
mt.row.alleles.show(5)

#### Calculate all possible GT calls over entire dataset:

There are no multiple variants, because you see no 2, 3 etc in output. This is because the data was prefiltered by TOPMed.

In [ ]:
possibleGT = mt.aggregate_entries(hl.agg.collect_as_set(mt.GT))
print(len(possibleGT))
possibleGT

#### Show unique possible calls occuring in entire dataset
- reference (ref) and alternate allele calls (alt).
- Contains homozygous reference, SNPs, indels, CNVs.

In [ ]:
unique_allelecalls = mt.aggregate_rows(hl.struct(
    ref = hl.agg.collect_as_set(mt.alleles[0]), 
    alt = hl.agg.collect_as_set(mt.alleles[1])))
pprint(unique_allelecalls)

# 2. Quality Control Variants

Initial QC:
- samples: sex, concordance with previous array data, relatedness (by TOPMed). 
- variants: SVM based on mendelian consistency (by University of Michigan).

### 2.1. Optional: Pruning variants in Linkage Disequilibrium
- Remove correlated variants from a matrix table. ld_prune() requires mt to contain only diploid GT calls.
- Windowsize for prunign follows the population specific haplotype block span. In African chromosomes 50% of genome lies in haplotype blocks > 22kb, and Europeans/Asians in >44 kb, because they descended from small number of ancestors who left Africa(Gabriel 2002 Science).
- However, the algorithm in Hail cannot handle such large data using a windowsize this small (Error summary: OutOfMemoryError: GC overhead limit exceeded). This was the case using 25 nodes of the type i3.4xlarge master and pre-empitble worker nodes with partitioned data (4 * 16 CPU * 25 nodes),  Hail version: 0.2.9-28ab341611d, bringing fs.s3.maxConnections wihtin the master node up to 5000 (see emr-timeout.sh); and waiting over 24hrs. 


In [ ]:
biallelic_mt = mt.filter_rows(hl.len(mt.alleles) == 2)

In [ ]:
pruned_t = hl.ld_prune(mt.GT, r2 = 0.8, bp_window_size = 44000,  memory_per_core = 128)

In [ ]:
mt = mt.filter_rows(hl.is_defined(pruned_t[mt.row_key]))

### 2.2.  Hardy-weinberg equilibrium
- HWE on only controls, separately for each ethnic group (Turner, 2011).
- The real difference is that Hail uses a mid-p correction on the exact test, but there should be no differences for small p-values (Graffelman et al., 2010). 

Filter for Caucasian population

In [9]:
mt_NHW = mt.filter_cols(mt.Race == "Caucasian") 

In [10]:
mt_AA = mt.filter_cols(mt.Race == "African American") 

In [11]:
mt_NHW = mt_NHW.annotate_rows(hwe_ctrl = hl.agg.filter(mt_NHW.Affection == 'Control', hl.agg.hardy_weinberg_test(mt_NHW.GT)))
mt_NHW = mt_NHW.filter_rows(mt_NHW.hwe_ctrl.p_value > 10**-5)


In [12]:
mt_AA = mt_AA.annotate_rows(hwe_ctrl = hl.agg.filter(mt_AA.Affection == 'Control', hl.agg.hardy_weinberg_test(mt_AA.GT)))
mt_AA = mt_AA.filter_rows(mt_AA.hwe_ctrl.p_value > 10**-5)


In [13]:
mt = mt_AA.union_cols(mt_NHW)

After HWE filter, 74164504 variants and 1876 samples remain.

### 2.3. Check for missingness
Calculate statistics per variant

Missing rate > 2%, thus keep variants with call rate > 0.98.

call_rate = called / (non-called + called).

In [ ]:
mt = hl.variant_qc(mt)

In [ ]:
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.98)

No variants were removed, call rate is 100% for all due to initial QC by TOPMed.

### 2.4. Remove duplicate IDs
Remove samples with same ID classifier.

In [ ]:
t = hl.rename_duplicates(mt).cols()

In [ ]:
t.key_by("s") 
mt.key_cols_by("s")

In [ ]:
mt = mt.filter_cols(hl.is_defined(t[mt.s]))

No variants were removed.

### 2.5. Remove wrongly assigned variant types
Genotype called homozygous reference with >10% alternate reads, a genotype called homozygous alternate with >10% reference reads, or a genotype called heterozygote without a ref / alt balance near 1:1, it is likely to be an error

In [ ]:
ab = mt.AD[1] / hl.sum(mt.AD)

filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))

mt = mt.filter_entries(filter_condition_ab)

No variants removed, due to prefiltering by TOPMed.

### 2.6. Remove non-autosomes
This step needs to be executed after the sex check for samples, because impute_sex() function needs the sex chromosome. You can find this step after the sample filtering steps.  

### 2.7. Optional: Keep SNPs (remove indels, CNVs etc.)

Comparing reference allele (0) to alternates (1)

In [ ]:
mt_snp = mt.filter_rows(hl.is_snp(mt.alleles[0], mt.alleles[1]))

Check SNPs: unique possible reference (ref) and alternate allele calls (alt) from entire dataset (all samples)

In [ ]:
unique_allelecalls = mt_snp.aggregate_rows(hl.struct(
    ref = hl.agg.collect_as_set(mt_snp.alleles[0]), 
    alt = hl.agg.collect_as_set(mt_snp.alleles[1])))
pprint(unique_allelecalls)

Check SNPs: shows all lenghts of vectors with possible allels (including ref, alternate)

In [ ]:
a = mt_snp.aggregate_rows(hl.agg.collect_as_set(hl.len(mt_snp.alleles)))
pprint(a)

In [ ]:
mt_AF = mt.filter_rows(mt.variant_qc.AF[1] >= 0.01)

# 3. Quality Control Samples 

### 3.1. Filter samples for outliers more than (6 * SD) from mean (part 1).
Calculate mean and SD per individual prior other sample filtering steps, because filtering affects the sample_qc(). 

The actual filtering is at step 3.5.



In [5]:
mt = hl.sample_qc(mt)

In [6]:
stats_singleton   = mt.aggregate_cols(hl.agg.stats(mt.sample_qc.n_singleton))
stats_ti_tv       = mt.aggregate_cols(hl.agg.stats(mt.sample_qc.r_ti_tv))
stats_het_hom_var = mt.aggregate_cols(hl.agg.stats(mt.sample_qc.r_het_hom_var))
stats_het         = mt.aggregate_cols(hl.agg.stats(mt.sample_qc.n_het))

2019-05-24 21:23:28 Hail: INFO: Coerced sorted dataset
2019-05-24 21:30:13 Hail: INFO: Coerced sorted dataset
2019-05-24 21:31:51 Hail: INFO: Coerced sorted dataset
2019-05-24 21:33:02 Hail: INFO: Coerced sorted dataset


### 3.2. Concordance 
Concordance of sequenced with previous genotyped/imputed data, however this data is not available. Concordance was only a problem in the version freeze 5 (and not freeze 4 what we are using here).

### 3.3. Sex check on chromosome X (inbreeding coefficient)
Remove samples where imputed genetic sex does not equal reported sex.

In [8]:
t = hl.impute_sex(mt.GT)

2019-05-24 21:33:03 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


In [9]:
mt = mt.filter_cols(t[mt.s].is_female == mt.is_female)

### 3.4. Check for genetic relationship / "duplicates"
- Calculate the Identity-By-Descent matrix: 
- Requires bi-allelic and preferably pruned input. 

In [ ]:
mt_relatedness = hl.identity_by_descent(mt)

- keep pairs of samples with PI_HAT in [0.2, 1] using MAF computed from the dataset itself in row field panel_maf.

In [ ]:
t_ibd = relatedness.filter(relatedness.ibd.PI_HAT > 0.2)

In [ ]:
t_ibd.key_by('i')

In [ ]:
mt.key_cols_by("s")

- Collect the IDs of the related samples in t_ibd

In [ ]:
ibd_idx = t_ibd.aggregate(hl.agg.collect_as_set(t_ibd.i))

In [ ]:
mt_ibd = mt.filter_cols(hl.is_defined(ibd_idx))

No samples were removed.

### 3.5. Filter on outliers:

####  Number of singletons

In [11]:
mt = mt.filter_cols(mt.sample_qc.n_singleton < (stats_singleton.mean + (6 * stats_singleton.stdev)))
mt = mt.filter_cols(mt.sample_qc.n_singleton > (stats_singleton.mean - (6 * stats_singleton.stdev)))

#### Ti/Tv ratio 

In [13]:
mt = mt.filter_cols(mt.sample_qc.r_ti_tv < (stats_ti_tv.mean + (6 * stats_ti_tv.stdev)))
mt = mt.filter_cols(mt.sample_qc.r_ti_tv > (stats_ti_tv.mean - (6 * stats_ti_tv.stdev)))

#### Het/hom ratio 
Only consider homologous variants, because homologous reference are filtered out.

In [15]:
mt = mt.filter_cols(mt.sample_qc.r_het_hom_var < (stats_het_hom_var.mean + (6 * stats_het_hom_var.stdev)))
mt = mt.filter_cols(mt.sample_qc.r_het_hom_var > (stats_het_hom_var.mean - (6 * stats_het_hom_var.stdev)))

#### Number of heterozygous calls

In [17]:
mt = mt.filter_cols(mt.sample_qc.n_het < (stats_het.mean + (6 * stats_het.stdev)))
mt = mt.filter_cols(mt.sample_qc.n_het > (stats_het.mean - (6 * stats_het.stdev)))

No samples were removed.

### 3.6. Remove non-autosomes(X, Y and MT DNA)
- Do this step after sex check

In [20]:
mt = mt.filter_rows(mt.locus.in_autosome())

### 3.7 Export QC filtered file

In [ ]:
mt.write("s3://yourpath", overwrite = True)

# 4. Characteristics filtered data

### 4.1 Number of SNPs, singletons, indels

- Total variants: 72083383
- SNP: 66419084 alternate alleles
- Deletion: 3980503 alternate alleles
- Insertion: 1683796 alternate alleles

In [22]:
hl.summarize_variants(mt)

Number of variants: 72083383
Alleles per variant
-------------------
  2 alleles: 72083383 variants
Variants per contig
-------------------
   1: 5719537 variants
   2: 6237105 variants
   3: 5128944 variants
   4: 5082138 variants
   5: 4707472 variants
   6: 4454090 variants
   7: 4222573 variants
   8: 4087631 variants
   9: 3224037 variants
  10: 3529399 variants
  11: 3538315 variants
  12: 3422240 variants
  13: 2527331 variants
  14: 2336668 variants
  15: 2154702 variants
  16: 2387231 variants
  17: 2082892 variants
  18: 2007482 variants
  19: 1657815 variants
  20: 1600922 variants
  21: 988639 variants
  22: 986220 variants
Allele type distribution
------------------------
        SNP: 66419084 alternate alleles
   Deletion: 3980503 alternate alleles
  Insertion: 1683796 alternate alleles


### 4.2. Subject Stats

Partition data into cases (mt_case) and controls (mt_ctrl) for African American and Caucasian populations and calculate for each the mean age, number of individuals, mean pack-years and mean FEV1%.

In [23]:
ca = mt.filter_cols(mt.Affection == 'Case')
co = mt.filter_cols(mt.Affection == 'Control')
ca_AA = ca.filter_cols(ca.Race == 'African American')
co_AA = co.filter_cols(co.Race == 'African American')
ca_CAU = ca.filter_cols(ca.Race == 'Caucasian')
co_CAU = co.filter_cols(co.Race == 'Caucasian')

In [ ]:
print('#Individuals of Cases:', mt_case.aggregate_cols(hl.agg.counter(mt_case.Race))) 
print('#Individuals of Controls:', mt_ctrl.aggregate_cols(hl.agg.counter(mt_ctrl.Race)))  

print('Age case AA =', ca_AA.aggregate_cols(hl.agg.stats(ca_AA.Age))) 
print('Age control AA =', co_AA.aggregate_cols(hl.agg.stats(co_AA.Age))) 
print('Age case CAU =', ca_CAU.aggregate_cols(hl.agg.stats(ca_CAU.Age))) 
print('Age control CAU =', co_CAU.aggregate_cols(hl.agg.stats(co_CAU.Age))) 

print('Gender case AA:', ca_AA.aggregate_cols(hl.agg.counter(ca_AA.is_female)))  
print('Gender control AA:', co_AA.aggregate_cols(hl.agg.counter(co_AA.is_female))) 
print('Gender case CAU:', ca_CAU.aggregate_cols(hl.agg.counter(ca_CAU.is_female)))  
print('Gender control CAU:', co_CAU.aggregate_cols(hl.agg.counter(co_CAU.is_female))) 

print("FEV1 case AA", ca_AA.aggregate_cols(hl.agg.stats(hl.float(ca_AA.FEV1perc))))
print("FEV1 control AA", co_AA.aggregate_cols(hl.agg.stats(hl.float(co_AA.FEV1perc))))
print("FEV1 case CAU", ca_CAU.aggregate_cols(hl.agg.stats(hl.float(ca_CAU.FEV1perc))))
print("FEV1 control CAU", co_CAU.aggregate_cols(hl.agg.stats(hl.float(co_CAU.FEV1perc))))

print("FEV1FVC case AA", ca_AA.aggregate_cols(hl.agg.stats(hl.float(ca_AA.FEV1FVC))))
print("FEV1FVC control AA", co_AA.aggregate_cols(hl.agg.stats(hl.float(co_AA.FEV1FVC))))
print("FEV1FVC case CAU", ca_CAU.aggregate_cols(hl.agg.stats(hl.float(ca_CAU.FEV1FVC))))
print("FEV1FVC control CAU", co_CAU.aggregate_cols(hl.agg.stats(hl.float(co_CAU.FEV1FVC))))

print('Pack-years of case AA =', ca_AA.aggregate_cols(hl.agg.stats(hl.float(ca_AA.PackYear)))) #
print('Pack-years of control AA =', co_AA.aggregate_cols(hl.agg.stats(hl.float(co_AA.PackYear)))) #
print('Pack-years of case CAU =', ca_CAU.aggregate_cols(hl.agg.stats(hl.float(ca_CAU.PackYear)))) #
print('Pack-years of control CAU =', co_CAU.aggregate_cols(hl.agg.stats(hl.float(co_CAU.PackYear)))) #

print('#Current Smoker case AA:', ca_AA.aggregate_cols(hl.agg.counter(ca_AA.CurrentSmoker)))  # M , F: }
print('#Current Smoker control AA:', co_AA.aggregate_cols(hl.agg.counter(co_AA.CurrentSmoker)))  # M , F: }
print('#Current Smoker case CAU:', ca_CAU.aggregate_cols(hl.agg.counter(ca_CAU.CurrentSmoker))) # M , F:
print('#Current Smoker control CAU:', co_CAU.aggregate_cols(hl.agg.counter(co_CAU.CurrentSmoker)))  # M , F: }



# 5. Correct for population stratification

Sample ancestry leads to a stratified distribution of the phenotype.

### 5.1 Common variants to preserve power. 
Similar to conventional EIGENSTRAT method.

In [7]:
mt = hl.variant_qc(mt)

In [8]:
mt_common = mt.filter_rows(mt.variant_qc.AF[1] > 0.05)

7.426.338 of 72.083.383 variants are common

#### Pruning

r2 = 0 means that variants are unrelated, and when 1 they are in full linkage disequilibrium. r = 0.5 means removing those with correlation between 0.5 and 1.

In [10]:
pruned_t = hl.ld_prune(mt_common.GT, r2 = 0.5, bp_window_size = 500000)


2019-05-29 13:25:27 Hail: INFO: ld_prune: running local pruning stage with max queue size of 137519 variants
2019-05-29 13:28:45 Hail: INFO: Wrote all 281 blocks of 1147272 x 1752 matrix with block size 4096.
2019-05-29 13:28:48 Hail: INFO: wrote table with 1147449 rows in 2560 partitions to hdfs://lindsay1-m/tmp/hail.fDj73JEwV5cC/kXHYqofZhB
2019-05-29 13:32:02 Hail: INFO: Wrote all 281 blocks of 1147449 x 1752 matrix with block size 4096.
2019-05-29 13:36:40 Hail: INFO: wrote table with 22896 rows in 561 partitions to hdfs://lindsay1-m/tmp/hail.fDj73JEwV5cC/A894GQPueT


After pruning, 1.129.238 variants

In [16]:
mt_pruned = mt_common.filter_rows(hl.is_defined(pruned_t[mt_common.row_key]))

### 5.3 Eigenvalues and PC scores

In [22]:
eigenvalues, scores, loadings = hl.hwe_normalized_pca(
    mt_common.GT, k = 10, compute_loadings = True)

2019-05-29 14:12:49 Hail: INFO: hwe_normalized_pca: running PCA using 1129328 variants.
2019-05-29 14:12:57 Hail: INFO: pca: running PCA with 10 components...
2019-05-29 14:14:40 Hail: INFO: Coerced sorted dataset


In [23]:
pprint(eigenvalues)

[159.4295821415391,
 2.2998149072116423,
 1.5611533801079325,
 1.541044253018341,
 1.539295927454413,
 1.5305640925963002,
 1.5201277241750528,
 1.5061649330313953,
 1.5019689834918581,
 1.4970116493813483]


In [24]:
scores.show(5, width = 100)

+-------------+
| s           |
+-------------+
| str         |
+-------------+
| "NWD100295" |
| "NWD100314" |
| "NWD100651" |
| "NWD101660" |
| "NWD101749" |
+-------------+

+--------------------------------------------------------------------------------------------------+
| scores                                                                                           |
+--------------------------------------------------------------------------------------------------+
| array<float64>                                                                                   |
+--------------------------------------------------------------------------------------------------+
| [-1.86e-01,-5.56e-05,4.23e-02,-1.55e-03,7.69e-02,-2.23e-02,2.51e-02,-3.92e-02,-2.72e-02,2.80e... |
| [3.00e-01,1.97e-02,8.74e-03,-5.86e-03,-1.35e-03,-4.46e-03,-3.85e-03,3.63e-03,-4.71e-03,-7.09e... |
| [-2.81e-01,5.02e-03,-8.55e-02,-5.37e-02,1.97e-02,4.75e-02,-1.84e-02,-7.98e-02,-3.07e-02,-4.28... |
| [2.99e-01,5.9

In [25]:
mt = mt.annotate_cols(scores = scores[mt.s].scores) 

In [38]:
pca = hl.plot.scatter(mt_common.scores[0],
                      mt_common.scores[1],
                      label = mt_common.Race,
                      title = 'PCA Caucasian', xlabel = 'PC1', ylabel = 'PC2')
show(pca)

2019-05-29 14:30:13 Hail: INFO: Coerced sorted dataset


Save in Google Dataproc:

In [ ]:
output_file('pca', title = "COPDfreeze4_pca" mode = 'cdn')
save(pca)
client.get_bucket('path_to_').blob('COPDfreeze4_pca.html').upload_from_filename('pca') 

### 5.4. Optional: Project new samples on existing PCs

In [ ]:
ef pc_project(
        # reference: https://github.com/macarthur-lab/gnomad_hail/blob/master/utils/generic.py#L131
        mt: hl.MatrixTable,
        loadings_ht: hl.Table,
        loading_location: str = "loadings",
        af_location: str = "pca_af"
) -> hl.Table:
    n_variants = loadings_ht.count()
    mt = mt.annotate_rows(
        pca_loadings=loadings_ht[mt.row_key][loading_location],
        pca_af=loadings_ht[mt.row_key][af_location]
    )
    mt = mt.filter_rows(hl.is_defined(mt.pca_loadings) & hl.is_defined(mt.pca_af) &
                        (mt.pca_af > 0) & (mt.pca_af < 1))
    gt_norm = (mt.GT.n_alt_alleles() - 2 * mt.pca_af) / hl.sqrt(n_variants * 2 * mt.pca_af * (1 - mt.pca_af))
    mt = mt.annotate_cols(scores=hl.agg.array_sum(mt.pca_loadings * gt_norm))
    return mt.cols().select('scores')
eigenvalues, scores, loadings = hl.hwe_normalized_pca(mt_common.GT, k = 10, compute_loadings = True)
mt_common = mt_common.annotate_cols(scores = scores[mt_common.s].scores)
mt_common = mt_common.annotate_rows(pca_af = hl.agg.mean(mt_common.GT.n_alt_alleles()) / 2)
loadings = loadings.annotate(pca_af = mt_common[loadings.key, :].pca_af)
related_scores = pc_project(related_mt, pca_loadings)

In [ ]:
eigenvalues, scores, loadings = hl.hwe_normalized_pca(
    mt_common.GT, k = 10, compute_loadings = True)

In [ ]:
mt_common = mt_common.annotate_cols(scores = scores[mt_common.s].scores) 

In [ ]:
mt_common = mt_common.annotate_rows(pca_af = hl.agg.mean(mt_common.GT.n_alt_alleles()) / 2)

In [ ]:
loadings = loadings.annotate(
    pca_af = mt_common[loadings.key, :].pca_af)

In [ ]:
related_scores = pc_project(related_mt, pca_loadings)

### 5.5. Optional: PCA manually

In [ ]:
x = pca_scores.scores[0]
y = pca_scores.scores[1]
label = mt.cols()[pca_scores.s].Race
collect_all = nullable(bool)

if isinstance(x, Expression) and isinstance(y, Expression):
        agg_f = x._aggregation_method()
        if isinstance(label, Expression):
            if collect_all:
                res = hail.tuple([x, y, label]).collect()
                label = [point[2] for point in res]
            else:
                res = agg_f(aggregators.downsample(x, y, label=label, n_divisions=n_divisions))
                label = [point[2][0] for point in res]

            x = [point[0] for point in res]
            y = [point[1] for point in res]
        else:
            if collect_all:
                res = hail.tuple([x, y]).collect()
            else:
                res = agg_f(aggregators.downsample(x, y, n_divisions=n_divisions))

            x = [point[0] for point in res]
            y = [point[1] for point in res]

            
arg = list(set(label))
color=[]
for i in label :
    if i == arg[1]:
        color.append('red')
    elif i == arg[2] :
        color.append('black')
    else : 
        color.append('blue')

In [ ]:
p = figure(title = 'PCA')
fields = dict(x = x, y = y, label = label, color = color)
source = ColumnDataSource(fields)
p.circle( x = 'x', y = 'y', legend = 'label', color = 'color', source = source)
show(p)

# 6. GWAS

### 6.1. GWAS table

Plots were made for common and rare, or common, or rare variants. In which common is > 0.05 (Gibson et al Common and rare variants: 20 arguments).

In [8]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

13438678 variants have MAF of 1% (rare and common) for 1752 participants

### 6.2. Firth logistic regression with Affection status
Correcting for:
- population stratification (population structure) with 2 JPCs within population.
- 1.0 is input variable number of alternate alleles, with input variable the genotype dosage derived from the PL field.
- pack-years
- Note: not for age, because it is known that age is positively linearly correlated to COPD severity, and in this data set the age is inverse to the case/ctrl, which would not benefit the regression.

In [ ]:
gwas = hl.logistic_regression_rows(
            test = "firth",
            y = hl.float(mt.AffectionBool),
            x = mt.GT.n_alt_alleles(),
            covariates = [1, hl.float(mt.PackYear), 
                          mt.scores[0], mt.scores[1],
                         mt.scores[2], mt.scores[3],
                         mt.scores[4], mt.scores[5],
                         mt.scores[6], mt.scores[7],
                         mt.scores[8], mt.scores[9]]) 

### 6.3. Linear regression with FEV1 or FEV1FVC variables (lung function)

In [ ]:
gwas_lin = hl.linear_regression_rows(
            y = hl.float(mt.FEV1FVC),
            x = mt.GT.n_alt_alleles(),
            covariates = [1, hl.float(mt.PackYear), 
                          mt.scores[0], mt.scores[1],
                         mt.scores[2], mt.scores[3],
                         mt.scores[4], mt.scores[5],
                         mt.scores[6], mt.scores[7],
                         mt.scores[8], mt.scores[9]]) 

### 6.4 QQ-plot

- Observed against expected p-value: variants that are further away from the diagonal line are more causally linked to COPD condition.
- To what extend was the correction of population stratification successful. 

In [68]:
qqplot = hl.plot.qq(gwas.p_value)

2019-06-07 17:12:43 Hail: INFO: Ordering unsorted dataset with network shuffle


In [69]:
show(qqplot)

Export for Google Dataproc

In [ ]:
output_file('qqplot', title = 'COPDfreeze4_qq', mode = 'cdn')
save(qqplot)
client.get_bucket('path_to_').blob('COPDfreeze4_qq.html').upload_from_filename('qqplot') 

### 6.5. Manhattan-like plots

- Chromosome position against p-value.
- GWAS significanse level = 5.0 * 10e-8, suggestive: 
Suggestive: 5.0 * 10e-8 < P < 5.0 * 10e-6.

Multiple testing bonferroni
- Bonferroni correction as a correction to the significance threshold, not the p-values.


In [79]:
Bonferroni_line = -np.log10(0.05 / mt.count_rows())

In [80]:
Suggestive_line = -np.log10(1 / mt.count_rows())

In [71]:
manh = hl.plot.manhattan(gwas.p_value, 
                         title = "Manhattan-like Plot", 
                         size = 4,
                         significance_line = 5e-08)

In [82]:
line1 = Span(location = Bonferroni_line, dimension = "width", line_color = "red", line_width = 1)
line2 = Span(location = Suggestive_line, dimension = "width", line_color = "orange", line_width = 1)
manh.renderers.extend([line1, line2])

In [98]:
manh.renderers.extend([line1, line2])

In [ ]:
show(manh)

Export for Google Dataproc

In [ ]:
output_file ('manh', title = 'COPDfreeze4_manh', mode = 'inline')
save(manh)
client.get_bucket('path_to_').blob('COPDfreeze4_manh.html').upload_from_filename('manh') 

### 6.6 Optional: FDR correction on P-values

Create pandas df of locus and P values from GWAS

In [ ]:
pval = gwas.select(gwas.locus, gwas.p_value)

In [ ]:
pval = gwas.select(gwas.locus, gwas.p_value)

In [ ]:
pval_pd = pval.to_pandas(flatten = True)

In [ ]:
pval = pval_pd.iloc[:,2]

Perform FDR correction on P-values

In [ ]:
def FDR(pvalues):
    pvalues = array(pvalues)
    n = len(pvalues)
    new_pvalues = empty(n)
    values = [ (pvalue, i) for i, pvalue in enumerate(pvalues) ]
    values.sort() # is already sorted
    values.reverse()
    new_values = []
    for i, vals in enumerate(values):
        rank = n - i
        pvalue, index = vals
        new_values.append((n/rank) * pvalue)
    for i in range(0, int(n)-1):
        if new_values[i] < new_values[i+1]:
            new_values[i+1] = new_values[i]
    for i, vals in enumerate(values):
        pvalue, index = vals
        new_pvalues[index] = new_values[i]
    return new_pvalues

In [ ]:
adjPval = FDR(pval)

In [ ]:
df = pd.concat([pval_pd, pd.DataFrame(adjPval)], axis = 1)

In [ ]:
df.rename(index = str, columns = {0:"adjPval"}, inplace = True)

Export to tsv format

In [ ]:
df.to_csv("/path_to_/adjPval.tsv", sep = "\t", header=True)

Convert to Hail table format

In [ ]:
df2 = hl.import_table("file:///path_to_/adjPval.tsv", 
                      types = {"locus.contig": hl.tstr,
                               "locus.position": hl.tint32,
                               "p_value": hl.tfloat64,
                               "adjPval": hl.tfloat64})

In [ ]:
df2 = df2.annotate(locus=hl.locus(df2["locus.contig"], df2["locus.position"]))

In [ ]:
df2 = df2.select(df2.locus, df2.adjPval)

Add adjusted P-values to MT file

In [ ]:
mt = mt.annotate_rows(**df2[mt.locus]) 

# 7. Tophits

## 7.1. Look into top hits from GWAS

In [87]:
gwas_ordered = gwas.order_by(gwas.p_value)

In [ ]:
gwas_ordered.p_value.show(25)

Check allele frequencies (pathogenic variants are expected to be low-frequency)

In [ ]:
mt_hits.variant_qc.AF[1].show()

Check which ones are SNPs

In [ ]:
mt_snp = mt_top.filter_rows(hl.is_snp(mt_top.alleles[0], mt_top.alleles[1]))

## 7.2. Filter on suggestive significance level

In [86]:
signPval = gwas.filter(gwas.p_value < 5e-06)

In [ ]:
n = signPval.count()

In [ ]:
n

In [ ]:
signPval.describe()

In [ ]:
signPval.show(n)

Filter MT file for selected variants

In [87]:
signPval = signPval.key_by("locus")

In [88]:
mt_hits = mt.filter_rows(hl.is_defined(signPval[mt.locus]))

For both african and american descent, a total of 89 variants were found at suggestive significance level.

They displayed the following allele frequency:

In [ ]:
mt_hits.variant_qc.AF[1].show(n)

### 7.3  (Optional: Prune hits)

The pruning algorithm finds correlation between genetic variants and keeps the first in chronological sequence order of the linkage disequilibrium block. This is likely not to be the most significant hit or most functionally important and for this reason was not incorporated in following analyses. 

In [ ]:
pruned_t = hl.ld_prune(mt_hits.GT, r2 = 0.1, bp_window_size = 1000000000)

27 variants were found out of linkage disequilibrium. 

In [8]:
mt_pruned = mt_hits.filter_rows(hl.is_defined(pruned_t[mt_hits.row_key]))

In [ ]:
mt_pruned.variant_qc.AF[1].show()

In [ ]:
mt_hits = mt_pruned

### Samples per top hits

Merge the data from the matrices globals, columns and rows into a single matrix

In [8]:
t = mt_hits.entries()

2019-06-07 19:52:16 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'


Check keys, because keys are kept anyways. 

In [9]:
t = t.key_by()

You also want to keep GT scores besides the cols locked as keys

In [11]:
t_GT = t.select(t.GT, t.dbGaP_Subject_ID, t.locus)

In [12]:
t_GT.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'GT': call 
    'dbGaP_Subject_ID': int32 
    'locus': locus<GRCh37> 
----------------------------------------
Key: []
----------------------------------------


In [13]:
t_pd = t_GT.to_pandas(flatten = True)

In [14]:
t_pd.to_csv('path_to_csv', encoding = 'utf-8', index = False)  #write a csv in the local directory
